# Exercise 1: Tool Calling

Large Language Models (LLMs) are incredibly powerful. However, as we've seen, they can struggle with simple mathematical problems and are limited by the information contained in their training data.

That said, we can describe tools, such as Python functions, to the LLM that may be useful for solving tasks. While the models themselves won't call the functions directly, they can determine when a function is needed and return the appropriate arguments for use.

In this notebook, you'll learn how describing functions to a model can transform it into an agent capable of reasoning with itself to answer a query.

In [21]:
# autoreload imports
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from llm_in_production.llm import instantiate_langchain_model
from llm_in_production.agent_utils import (
    # function_to_json,
    tool_calling_agent,
)
from langchain_core.utils.function_calling import convert_to_openai_function
import dotenv
import numpy as np
import os
import json
import requests
import pandas as pd
import yfinance as yf
import tiktoken

# This reads the .env file in your project and transforms its content into env variables.
# This way you don't have to hard code your secrets.
dotenv.load_dotenv()

# Here we create the client. 
# Make sure you select the LLM provider that corresponds to the one you are using in this course!
client = instantiate_langchain_model(
    # llm_provider="azure",
    llm_provider="gcp",
)
client.model_name

'gemini-1.5-flash'

## LLM limiations

Let's start by demonstrating one of the limitations of LLMs.

The prompt below is asking the model to answer a simple highschool mathematics problem.

In [3]:
math_problem = "what is the circumference of a circle with radius 5.31cm"

In [4]:
system_prompt = "You are a calculator bot that is used to answer mathematics problems"

In [5]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": math_problem},
]


In [6]:

response = client.invoke(
    input=messages,
    seed=0,
)

message = response.content

print(message)

The circumference of a circle is calculated using the formula:

**Circumference = 2 * π * radius**

Where:

* **π (pi)** is a mathematical constant approximately equal to 3.14159
* **radius** is the distance from the center of the circle to its edge.

In this case, the radius is 5.31 cm. 

So, the circumference is:

Circumference = 2 * 3.14159 * 5.31 cm 
**Circumference ≈ 33.38 cm** 



However, the answer it produces is incorrect.

$2*\pi*5.31 = 33.3637139811...$

For this problem, a simple Python function would be more useful!

In [7]:
def circumference_calculator(radius: float, something: float = 4.4) -> float:
    """Calculates the circumference of a circle given the radius

    :param radius: The radius of the circle
    :return: The circumference of the circle
    """
    return 2 * np.pi * radius


circumference_calculator(5.31)

33.3637139811236

But what if the LLM knew about this Python function?

## Tool Calling

It is now possible to inform LLMs about external tools.

The model can then determine on its own when it's appropriate to use one of these tools and, if needed, return the appropriate arguments.

To describe a function to the model it must be in a particular JSON format. LangChain have created a helpful function to transform Python functions into this format.

In [8]:
circumference_calculator_json = convert_to_openai_function(circumference_calculator)
circumference_calculator_json

{'name': 'circumference_calculator',
 'description': 'Calculates the circumference of a circle given the radius :param radius: The radius of the circle\n:return: The circumference of the circle',
 'parameters': {'properties': {'radius': {'type': 'number'},
   'something': {'default': 4.4, 'type': 'number'}},
  'required': ['radius'],
  'type': 'object'}}

Let's give the model access to this tool and repeat the prompt we did before.

In [9]:
tools = [{"type": "function", "function": circumference_calculator_json}]

response = client.invoke(
    input=messages,
    seed=0,
    tools=tools,
    tool_choice = "auto"  # (default setting) the model will pick between generating a message or calling a function automatically 
)
response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'circumference_calculator', 'arguments': '{"radius": 5.31}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.09521484375, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.064453125}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.10107421875, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.07177734375}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.16015625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.11572265625}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.06640625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.02

The model has no message to return:

In [10]:
message = response.content
print(f"Message: {message}")

Message: 


But, it has called for a tool to be used, with arguments!

In [11]:
tool_calls = response.tool_calls
print(f"Tool calls: {tool_calls}")

Tool calls: [{'name': 'circumference_calculator', 'args': {'radius': 5.31}, 'id': '02195e83-21cd-4ea0-806b-74d84ab3e573', 'type': 'tool_call'}]


Let's do what the model suggests and call the Python function `circumference_calculator`, with the argument `radius=5.31`.

In [12]:
tool_name = tool_calls[0]['name']
tool_args = tool_calls[0]['args']
tool_response = json.dumps(eval(tool_name)(**tool_args))

print(f"Function response {tool_response}")

Function response 33.3637139811236


The number is correct. Let's pass information back to the model.

First we need to update the message history.

In [13]:
messages.append(response)

messages.append(
    {
        "role": "tool",
        "name": tool_name,
        "content": tool_response,
        "tool_call_id": tool_calls[0]['id'],
    }
)  # extend conversation with function response

if len(messages) > 4:
    raise Exception(
        "Too many messages have been added! Restart and rerun the notebook."
    )

messages

[{'role': 'system',
  'content': 'You are a calculator bot that is used to answer mathematics problems'},
 {'role': 'user',
  'content': 'what is the circumference of a circle with radius 5.31cm'},
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'circumference_calculator', 'arguments': '{"radius": 5.31}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.09521484375, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.064453125}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.10107421875, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.07177734375}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.16015625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.11572265625

Now we can call the model again to answer the original query.

In [14]:
response = client.invoke(
    input=messages,
    seed=0,
    tools=tools,
)  # get a new response using the result of the function call


message = response.content
tool_calls = response.tool_calls

print(f"New tool calls: {len(tool_calls)}")
print(message)

New tool calls: 0
The circumference of the circle is 33.3637139811236 cm. 



Thanks to tool calling, we were able to produce the correct result!

Notice how the model will only reach for a function if it is needed.

In [16]:
history_problem = "How many wives did king Henry VIII of England have?"

response = client.invoke(
    input=[
        # {"role": "system", "content": system_prompt},
        {"role": "user", "content": history_problem},
    ],
    seed=0,
    tools=tools,
)

message = response.content
tool_calls = response.tool_calls

print(f"Message: {message}")
print(f"Tool calls: {tool_calls}")

Message: King Henry VIII of England had six wives. 

Tool calls: []


## External APIs

One way of gathering live information is through the use of an external API.

They can enable us to retrieve a wide variety of information (although some will come at a cost for commercial usage). For example:
- The weather conditions for a particular location.
- The topic headline from a particular news source.
- The recent prices of a stock .


### Weather

To get information about local weather we can use the API from https://www.weatherapi.com, which is availble on a [free plan](https://www.weatherapi.com/pricing.aspx). An API key has been provided for you in the Codespace.

We can use the function below to obtain weather information for a particular location. You can take a look at the [WeatherAPI docs](https://www.weatherapi.com/docs/) for more examples of how to use the API.

In [30]:
from dotenv import load_dotenv

load_dotenv()

%reload_ext dotenv
%dotenv /Users/dpranantha/Projects/bol/training/academy-llm-applications-course/.env

def get_current_weather(location: str) -> dict:
    """Get the current weather conditions in a given location

    :param location: The city (and state), e.g. "San Francisco, CA"
    :return: The weather conditions
    """
    return requests.get(
        f"https://api.weatherapi.com/v1/current.json?key=4aebfed4097a43aebe6135356250603&q={location}"
    ).json()


get_current_weather("London")

TODO


{'location': {'name': 'London',
  'region': 'City of London, Greater London',
  'country': 'United Kingdom',
  'lat': 51.5171,
  'lon': -0.1062,
  'tz_id': 'Europe/London',
  'localtime_epoch': 1741269563,
  'localtime': '2025-03-06 13:59'},
 'current': {'last_updated_epoch': 1741268700,
  'last_updated': '2025-03-06 13:45',
  'temp_c': 17.1,
  'temp_f': 62.8,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 12.1,
  'wind_kph': 19.4,
  'wind_degree': 185,
  'wind_dir': 'S',
  'pressure_mb': 1012.0,
  'pressure_in': 29.88,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 45,
  'cloud': 0,
  'feelslike_c': 17.1,
  'feelslike_f': 62.8,
  'windchill_c': 12.8,
  'windchill_f': 55.0,
  'heatindex_c': 14.3,
  'heatindex_f': 57.7,
  'dewpoint_c': 4.4,
  'dewpoint_f': 39.9,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 2.3,
  'gust_mph': 14.1,
  'gust_kph': 22.7}}

There is a lot of information in the JSON that the function returns, more than is needed for the average person. However, an LLM would be able to summarize this information into a readable paragraph.

In the cell below, we have written a system prompt and user prompt, which we pass to the `tool_calling_agent` function, along with the `get_current_weather` function.

The `tool_calling_agent` is a helpful function we wrote to automate the reasoning process used above:

- First, any functions or tools we want to give the model access to are transformed into the JSON format it expects.
- We then extract a response from the model based on the input prompts.
- The process may finish there, but if the model deems it necessary, it may ask for a tool to be called. In this case:
    - The tool is called using the parameters returned by the model.
    - The tool’s response is then given to the model.
    - Finally, the output based on this new information is returned.
    
We’ve added some logging to help you understand what’s going on "under the hood."


In [31]:
system_prompt = """
You are a weather bot that is able to give back a summary of the weather conditions in the given location. 

You do not return many measurements about the conditions.

Instead, you give an overall idea as to what conditions are like in one or two sentences."
"""

user_prompt = "What is the weather in London?"

In [32]:
output = tool_calling_agent(client, system_prompt, user_prompt, get_current_weather)
print(output)


### Model Output:



### Tool Called: get_current_weather with args: {'location': 'London'}

### Model Output:

It is sunny in London with a temperature of 17.1 degrees Celsius. 




### Exercise 1a: News

Your task is to build an agent that can summarize the top headlines of the day from a given news source.

To give the model access to the latest news stories, you can use the https://newsapi.org/ API, which is available on a [free plan](https://newsapi.org/pricing). An API key is already provided for you in the Codespace.


#### Part i: The day's top headlines

Write a Python function that returns JSON information about the top headlines of the day (from a given source) in the Python dictionary format. You can use the examples given in the [NewsAPI docs](https://newsapi.org/docs/endpoints/top-headlines) to help you answer this question.

In [51]:
def get_top_headlines(source: str) -> dict:
    # YOUR CODE HERE START
    return requests.get(
        f"https://newsapi.org/v2/top-headlines?apiKey=a49e3df725554d18b2ad712959e8fd61&source={source}&category=politics"
    ).json()
    # YOUR CODE HERE END

get_top_headlines("bbc-news")

{'status': 'ok',
 'totalResults': 35,
 'articles': [{'source': {'id': None, 'name': 'MarketWatch'},
   'author': 'Jeffry Bartash, Myra P. Saefong, Vivien Lou Chen, Joseph Adinolfi, Isabel Wang, Christine Idzelis, Joy Wiltermuth, Jamie Chisholm, Steve Goldstein',
   'title': 'Stock Market Today: Dow, S&P 500 and Nasdaq futures erase gains as investors weigh weak ADP report, potential tariff relief - MarketWatch',
   'description': 'Stocks are struggling to hold an early bounce Wednesday, with focus on weaker-than-forecast ADP jobs data and hopes for a potential softer approach on tariffs from President Trump.',
   'url': 'https://www.marketwatch.com/livecoverage/stock-market-today-dow-s-p-500-and-nasdaq-in-line-for-rebound-amid-volatile-trading',
   'urlToImage': 'https://images.mktw.net/im-38813246/social',
   'publishedAt': '2025-03-05T14:20:00Z',
   'content': None},
  {'source': {'id': 'cnn', 'name': 'CNN'},
   'author': 'John Fritze',
   'title': 'Supreme Court rejects Trump’s requ

#### Part ii: Prompt engineering for a news summary

Write a system prompt and user prompt that will lead the agent to summarize the day's top headlines. 

Rather than simply returning a list of the top stories, the agent should be able to condense the information into a few sentences. The agent should also be capable of only writing about the news category the user is interested in, e.g. politics or sport.

In [52]:
# YOUR CODE HERE START
system_prompt = """
You are a newscaster bot that is able to give back a summary of the top stories from a news source in the given category selected by a user.

You do not return the top stories per se.

Instead, you need to condense the overall information into a few sentences from those top stories and only within that category of interest.
"""

user_prompt = "What is the state of politic affairs worldwide from BBC news?"
# YOUR CODE HERE END

In [53]:
output = tool_calling_agent(client, system_prompt, user_prompt, get_top_headlines)
print(output)


### Model Output:



### Tool Called: get_top_headlines with args: {'source': 'bbc-news'}

### Model Output:

The US and Israel have rejected an Arab plan for the reconstruction of the Gaza Strip. The US has paused intelligence support and weapons shipments to Ukraine following a clash between President Zelensky and President Trump. The Ukrainian leader's climb-down on security guarantees may be because the US is the only place he can turn to.
```



### Exercise 1b: Stocks

In this exercise you will build a stock analysis agent to help a user to make investment decisions.

The agent will be able examine the recent history of a stock, along with recent press articles about it, to determine whether or not the stock is a good investment.

<mark>Warning! This agent is only intended to be used for educational purpose and we do not expect its suggestions to be of much worth.</mark> 

<mark> Xebia will not be held liable for any financial losses incurred from its usage!</mark> 

To get the recent history of a stock we will use [yfinance](https://github.com/ranaroussi/yfinance). yfinance offers a threaded and Pythonic way to download market data from [Yahoo!Ⓡ finance](https://finance.yahoo.com). Note that it is **not** affiliated, endorsed, or vetted by Yahoo, Inc. It's an open-source tool that uses Yahoo's publicly available APIs, and is intended for research and educational purposes.

Given a stock ticker symbol, we can retrive recent information about that stock.

In [44]:
stock = yf.Ticker("IBM")
df = stock.history(period="1mo")
df = df[["Close", "Volume"]]
print(df.index.max() - df.index.min())
df.index = [str(x).split()[0] for x in list(df.index)]
df.index.rename("Date", inplace=True)
df

27 days 00:00:00


,Close,Volume
Date,,
2025-02-06,251.762711,6128300
2025-02-07,250.669983,3370300
2025-02-10,249.270004,3564400
2025-02-11,254.699997,4801600
2025-02-12,255.809998,3075300
2025-02-13,259.190002,4531500
2025-02-14,261.279999,3925300
2025-02-18,263.070007,4262800
2025-02-19,264.320007,3718700


#### Part i: Recent stock history

Write a Python function that returns JSON information about the recent history of a particular stock in the Python dictionary format.

In [56]:
def get_stock_prices(company_stock_ticker_symbol: str, period: str = "1mo") -> dict:
    # YOUR CODE HERE START
    stock = yf.Ticker(company_stock_ticker_symbol)
    if stock:
        df = stock.history(period="1mo")
        df = df[["Close", "Volume"]]
        df.index = [str(x).split()[0] for x in list(df.index)]
        df.index.rename("Date", inplace=True)
# YOUR CODE HERE END
        return df.to_json()
    else:
        {}

In [57]:
get_stock_prices("IBM")

'{"Close":{"2025-02-06":251.7627105713,"2025-02-07":250.6699829102,"2025-02-10":249.2700042725,"2025-02-11":254.6999969482,"2025-02-12":255.8099975586,"2025-02-13":259.1900024414,"2025-02-14":261.2799987793,"2025-02-18":263.0700073242,"2025-02-19":264.3200073242,"2025-02-20":264.7399902344,"2025-02-21":261.4800109863,"2025-02-24":261.8699951172,"2025-02-25":257.75,"2025-02-26":255.8399963379,"2025-02-27":253.2299957275,"2025-02-28":252.4400024414,"2025-03-03":250.1900024414,"2025-03-04":253.2100067139,"2025-03-05":251.3500061035},"Volume":{"2025-02-06":6128300,"2025-02-07":3370300,"2025-02-10":3564400,"2025-02-11":4801600,"2025-02-12":3075300,"2025-02-13":4531500,"2025-02-14":3925300,"2025-02-18":4262800,"2025-02-19":3718700,"2025-02-20":4884800,"2025-02-21":5667900,"2025-02-24":4398100,"2025-02-25":6292200,"2025-02-26":3460100,"2025-02-27":3402200,"2025-02-28":7988800,"2025-03-03":2977700,"2025-03-04":5342100,"2025-03-05":4009100}}'

We can use NewsAPI again to retrieve the recent headlines about a particular stock.

In [58]:
today = pd.Timestamp.today()
start = pd.Timestamp.today()
end = today - pd.Timedelta(days=30)

In [60]:
topic = "IBM stock news"

request = requests.get(
    f"https://newsapi.org/v2/everything?q={topic}&from={start}&to={end}&sortBy=popularity&apiKey=a49e3df725554d18b2ad712959e8fd61"
).json()
articles = pd.DataFrame(request["articles"])
titles = articles["title"]
titles


0     Aberdeen restores the 'e's after admitting 'ab...
1     IBM’s Arvind Krishna Is Betting on Specialized AI
2     Work at the Mill Or, the Story of Digital Equi...
3     The Tinkerings of Robert Noyce by Tom Wolfe (1...
4     International Business Machines Corporation (I...
5     Intel's vacant CEO spot rumored to be filled b...
6     Vulnerability Summary for the Week of February...
7     On theCUBE Pod: IBM stock soars as DeepSeek ne...
8     Nvidia’s fine! Besides, who else is going to p...
9     DeepSeek didn’t deep-six AI startup funding af...
10    As Trump and Musk scramble everything, investo...
11    Dave Vellante’s Breaking Analysis: The complet...
12    Dave Vellante’s Breaking Analysis: The complet...
13    Dave Vellante’s Breaking Analysis: The complet...
14    Das Cyber-Vorhängeschloss knacken: Warum Quant...
15    Stock Market Live Updates 21 February 2025: Se...
16    GitHub - modelcontextprotocol/servers: Model C...
17    IBM CEO Arvind Krishna: AI Adoption Will ‘

#### Part ii: Any recent news?

Write a Python function that returns JSON information about the recent headlines of a particulaer stock in the Python dictionary format.

In [61]:
def get_news_stories(topic: str) -> dict:
    # YOUR CODE HERE START
    request = requests.get(
        f"https://newsapi.org/v2/everything?q={topic}&from={start}&to={end}&sortBy=popularity&apiKey=a49e3df725554d18b2ad712959e8fd61"
    ).json()
    # YOUR CODE HERE END
    return titles.to_json()

In [62]:
get_news_stories("Apple")

'{"0":"Aberdeen restores the \'e\'s after admitting \'abrdn\' rebrand wasn\'t that clvr","1":"IBM\\u2019s Arvind Krishna Is Betting on Specialized AI","2":"Work at the Mill Or, the Story of Digital Equipment Corporation","3":"The Tinkerings of Robert Noyce by Tom Wolfe (1983)","4":"International Business Machines Corporation (IBM) Reaffirmed with Buy Rating by Bank of America, Set Target at $270 Following Investor Day","5":"Intel\'s vacant CEO spot rumored to be filled by Toms Caulfield \\u2014 abrupt GlobalFoundries shakeup sparks speculation","6":"Vulnerability Summary for the Week of February 10, 2025","7":"On theCUBE Pod: IBM stock soars as DeepSeek news continues to shake up industry","8":"Nvidia\\u2019s fine! Besides, who else is going to power all these new AI models?","9":"DeepSeek didn\\u2019t deep-six AI startup funding after all","10":"As Trump and Musk scramble everything, investors cool on enterprise tech\\u2019s prospects","11":"Dave Vellante\\u2019s Breaking Analysis: Th

#### Part iii: Prompt engineering for a stock

Write a system prompt and user prompt that will lead the agent to determine whether or not a stock is a good investment. 

As you may have already experienced with ChatGPT, the model will want to caveat its opinions.
For the purpose of this assignment, see if you can get it to provide an opinion without any reservations.

In [63]:
# YOUR CODE HERE START
system_prompt = """
You are a stockbroker bot that is able to give advise or opinion if someone should invest in a stock of a company based on the stock prices and news about that company and news source given by a user. The company name is the topic for the news source.

You do not return the top stories and stock price per se.

Instead, you need to condense the overall information into a few sentences and provide opinion of yes or no.
"""

user_prompt = "Please read BBC if I should invest on Apple?"
# YOUR CODE HERE END

In [64]:
output = tool_calling_agent(client, system_prompt, user_prompt, get_news_stories, get_stock_prices)
print(output)


### Model Output:



### Tool Called: get_news_stories with args: {'topic': 'Apple'}

### Model Output:

```


### Tool Called: get_stock_prices with args: {'company_stock_ticker_symbol': 'AAPL'}

### Model Output:

Apple stock has been on a steady rise recently, with a slight dip in the last few days. The news is mostly positive, with articles about IBM's AI adoption and the potential for AI to drive growth in the tech sector. Overall, the outlook for Apple is positive. 
Yes.




#### Part iv: Prompt engineering for multiple stocks.

Update your system/user prompt so that it will lead the agent to make an investment decision for more than one stock.

In [71]:
# YOUR CODE HERE START
system_prompt = """
You are a stockbroker bot that is able to give advise or opinion if someone should invest in stocks of several companies based on the stock prices and news about those companies and news source given by a user. The company name is the topic for the news source.

You do not return the top stories and stock prices per se.

Instead, you need to condense the overall information into a few sentences and provide opinion of yes or no for each company asked. Also provide alternative of companies.
"""

user_prompt = "Please read BBC if I should invest on Apple, Palantir, NVIDIA and Google?"
# YOUR CODE HERE END

In [72]:
output = tool_calling_agent(client, system_prompt, user_prompt, get_news_stories, get_stock_prices)
print(output)


### Model Output:



### Tool Called: get_news_stories with args: {'topic': 'Apple'}

### Tool Called: get_news_stories with args: {'topic': 'Palantir'}

### Tool Called: get_news_stories with args: {'topic': 'NVIDIA'}

### Tool Called: get_news_stories with args: {'topic': 'Google'}

### Tool Called: get_stock_prices with args: {'company_stock_ticker_symbol': 'AAPL'}

### Tool Called: get_stock_prices with args: {'company_stock_ticker_symbol': 'PLTR'}

### Tool Called: get_stock_prices with args: {'company_stock_ticker_symbol': 'NVDA'}

### Tool Called: get_stock_prices with args: {'company_stock_ticker_symbol': 'GOOGL'}

### Model Output:

Apple is doing well, but the stock price is a bit high.  I would say no.  Consider investing in Palantir or NVIDIA.  Palantir is a data analytics company that is growing rapidly.  NVIDIA is a graphics processing unit (GPU) company that is benefiting from the growth of artificial intelligence (AI).  Google is a good company, but the stock price is 

---